#  Rosenbrock function
---
Description:

- Optimization (min)
- Single-objective
- Constraints (1)
---

The general equation is given by:

- $f(x, y) = (1 - x)^2 + 100(y - x^2)^2$, with $-1.5 \le x \le +1.5$ and $-1.5 \le y \le +1.5$.

The problem here is that we are trying to minimize this function subject to the following constraint:

- $x^2 + y^2 \le 2$.

To do this we apply the [Penalty method](https://en.wikipedia.org/wiki/Penalty_method). Within this setting the global minimum is found at:

$\hat{f}(1.0, 1.0) = 0.0$.

### First we import python libraries and set up the directory of our code.

In [3]:
import os, sys
import numpy as np

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [5]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.standard_ga import StandardGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.meta_mutator import MetaMutator

### Define the Rosebrock function, which plays also the role of the 'fitness' function.

In addition, we define the 'rand_fx' which takes the role of the 'random()' method of the Genes. Every time we
want to 'mutate' a gene this function will be called that returns 'valid', but random values for the gene.

In [7]:
# Rosenbrock function.
def fun_Rosenbrock(individual: Chromosome, f_min: bool = True):
    
    # Penalty coefficient.
    p_coeff = 0.5
    
    # Extract the data values as 'x' and 'y', for parsimony.
    x, y = [gene.value for gene in individual.genome]
    
    # Compute the function value.
    f_val = (1.0 - x)**2 + 100.0*(y - x**2)**2
    
    # Compute the constraint.
    f_const = p_coeff*max(0.0, x**2 + y**2 - 2.0)**2
    
    # NOTE: the constraint is added with the penalty coefficient.
    return -(f_val + f_const) if f_min else f_val
# _end_def_

# Random function: ~U(-1.5, +1.5).
rand_fx = lambda: np.random.uniform(-1.5, +1.5)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each
gene has the same random() function (set by 'rand_fx'). But if the problem demands otherwise it is easy to set a 
different random() function for each gene.

In [9]:
# Define the number of chromosomes.
N = 100

# Initial population.
population = [Chromosome([Gene(np.random.normal(), rand_fx),
                          Gene(np.random.normal(), rand_fx)], np.nan, True)
              for _ in range(N)]

# Create the StandardGA object that will carry on the optimization.
test_GA = StandardGA(initial_pop=population,
                     fit_func=fun_Rosenbrock,
                     select_op=LinearRankSelector(),
                     mutate_op=MetaMutator(0.5),
                     crossx_op=UniformCrossover())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter,
such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [11]:
test_GA(epochs=5000, elitism=True, verbose=False)

Initial Avg. Fitness = -423.5807.
Final   Avg. Fitness = -29.4626.
Elapsed time: 55.805 seconds.


In [12]:
# Extract the optimal solution from the GA.
optimal_solution = test_GA.best_chromosome()

# Display the (final) optimal value.
print(f"Minimum Found: {fun_Rosenbrock(optimal_solution, f_min=False):.5f}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.genome):
    print(f"x{i} = {xi.value:.5f}")
# _end_for_

# True minimum: f(1.0, 1.0) = 0.0

Minimum Found: 0.00000

x0 = 0.99816
x1 = 0.99626


In [13]:
# If we want we can also print some operator statistics.
test_GA.print_operator_stats()

 LinearRankSelector: (140715408019584)
 _probability: 1.0
 _counter: 5000
 _lock: <unlocked _thread.lock object at 0x7ffadbe952c0>
 _items: None

 UniformCrossover: (140715408020224)
 _probability: 0.9
 _counter: 225178
 _lock: <unlocked _thread.lock object at 0x7ffadb6fa240>
 _items: None

 MetaMutator: (140715408021264)
 _probability: 0.5
 _counter: 250369
 _lock: <unlocked _thread.lock object at 0x7ffadbe95580>
 _items: (SwapMutator(1.0), RandomMutator(1.0), ShuffleMutator(1.0), InverseMutator(1.0))

 SwapMutator: (140715408020144)
 _probability: 1.0
 _counter: 62660
 _lock: <unlocked _thread.lock object at 0x7ffadbe80cc0>
 _items: None

 RandomMutator: (140715408021344)
 _probability: 1.0
 _counter: 62590
 _lock: <unlocked _thread.lock object at 0x7ffadbe95980>
 _items: None

 ShuffleMutator: (140715408020464)
 _probability: 1.0
 _counter: 62259
 _lock: <unlocked _thread.lock object at 0x7ffadb04f340>
 _items: None

 InverseMutator: (140715405303120)
 _probability: 1.0
 _counter: 6

### End of file